# Data Preprocess - 현재 사용 X, WIP, 좀 더 맨정신일떄

## 목표
+ 분할된 데이터를 원하는 형태로 변환
+ 최적화를 위한 전처리 작업

In [22]:
import pandas as pd
import os
import datetime

일단은 미니 코스피 부터 처리하자. 
현재 미니 코스피의 데이터는

&nbsp;|date|	time	|close|	volume	|sell|	buy
---|---|---|---|---|---|---
2018-11-22| 9:00	|20181122	|900	|268.5599976	|29|	0	|29
2018-11-22 |9:00	|20181122	|900|	268.6000061|	1	|1|	0

이렇게 구성되어있는데, 첫번째 col이 timestamp 양식을 따르니까 datetime으로 weekly 단위로 추출가능 할듯. 

일단 파일 A를 읽으면 그 A안에서 주간 단위로  $P[b_t = 1] $ 랑 $P[b_t = 0]$ 를 담은 Dataframe을 생성하자.  
이렇게하면  Parallel 하게 연산할 수 있는 부분을 미리 해둬서 전체 연산시간을 줄일 수 있을 듯.

+ 전체 csv를 하나로 합쳐서 하는것도 생각해봤는데, 장기적으로 메모리 이슈 생각해보면 파일을 분할해서 사용하는 게 좋을 거라 생각

In [26]:
'''
path = 'MiniKospi'
list_file = os.listdir(path)
for data in list_file :
'''
# 일단 파일 하나 부터
tmp  = pd.read_csv('MiniKospi/181122-1219.csv',index_col=0) 

tmp.index = pd.to_datetime(tmp.index)
tmp.index.names = ['timestamp']
display(tmp.head())

,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-22 09:00:00,20181122,900,268.559998,29.0,0.0,29.0
2018-11-22 09:00:00,20181122,900,268.600006,1.0,1.0,0.0
2018-11-22 09:00:00,20181122,900,268.579987,1.0,0.0,1.0
2018-11-22 09:00:00,20181122,900,268.660004,1.0,1.0,0.0
2018-11-22 09:00:00,20181122,900,268.619995,1.0,0.0,1.0


주 단위로 나눠보자

https://stackoverflow.com/questions/41625077/python-pandas-split-a-timeserie-per-month-or-week  

[Grouper](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html)

In [13]:
weeks = [g for n, g in tmp.groupby(pd.Grouper(level='timestamp', freq='W'))]
type(weeks)

list

In [19]:
# 확인
display(weeks[0].head())
display(weeks[0].tail())
display(weeks[1].head())
display(weeks[1].tail())

,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-22 09:00:00,20181122,900,268.559998,29.0,0.0,29.0
2018-11-22 09:00:00,20181122,900,268.600006,1.0,1.0,0.0
2018-11-22 09:00:00,20181122,900,268.579987,1.0,0.0,1.0
2018-11-22 09:00:00,20181122,900,268.660004,1.0,1.0,0.0
2018-11-22 09:00:00,20181122,900,268.619995,1.0,0.0,1.0


,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-23 15:34:00,20181123,1534,266.500000,11.0,11.0,0.0
2018-11-23 15:34:00,20181123,1534,266.519989,1.0,1.0,0.0
2018-11-23 15:34:00,20181123,1534,266.500000,1.0,1.0,0.0
2018-11-23 15:34:00,20181123,1534,266.519989,2.0,0.0,2.0
2018-11-23 15:45:00,20181123,1545,266.359985,81.0,0.0,81.0


,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-26 09:00:00,20181126,900,266.940002,234.0,0.0,234.0
2018-11-26 09:00:00,20181126,900,266.959991,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,266.980011,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,266.980011,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,267.000000,135.0,0.0,135.0


,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-30 15:34:00,20181130,1534,271.720001,4.0,0.0,4.0
2018-11-30 15:34:00,20181130,1534,271.679993,10.0,10.0,0.0
2018-11-30 15:34:00,20181130,1534,271.660004,1.0,1.0,0.0
2018-11-30 15:34:00,20181130,1534,271.660004,1.0,1.0,0.0
2018-11-30 15:45:00,20181130,1545,271.559998,50.0,0.0,50.0


주단위 - 5일 - 로 나눠진것을 볼수 있다. 
첫주는 오롯한 5일이 아닐 수 있기 때문에 사용하지 않을 생각이다. 

이제 
$P[b_t =1]$ , $P[b_t =0]$ 을 구해보자. 

t=1 일 때의 기준은 시작가로. 

In [23]:
threshold = 1000

week_df = pd.Dataframe(index='date',colmuns=['p_t'])

cur_tick = 0

cur_p = weeks[1].loc[]['close']
prev_p = 0
d_p=0

for i in weeks[1].index:
    cur_tick += 1
    if c_tick = threshold : 
        prev_p = cur_p
        cur_p  = weeks[1].loc[i]['close']
        d_p = cur_p - prev_p
        
        n_t +=1
    b_t=[]

,date,time,close,volume,sell,buy
timestamp,,,,,,
2018-11-26 09:00:00,20181126,900,266.940002,234.0,0.0,234.0
2018-11-26 09:00:00,20181126,900,266.959991,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,266.980011,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,266.980011,1.0,0.0,1.0
2018-11-26 09:00:00,20181126,900,267.000000,135.0,0.0,135.0
